# ResNet 구현

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as T
import torchvision.utils as vutils
import torch.nn.functional as F

from torchsummary import summary

### BottleNeck Module

In [6]:
class BottleNeck(torch.nn.Module): # bottleneck buliding block
    def __init__(self, in_channels, out_channels, stride=1, mul=4):
        super(BottleNeck, self).__init__()
        self.mul = mul # 출력 채널 수를 몇 배로 늘릴 것인지
        #첫 Convolution은 너비와 높이 downsampling
        self.conv1 = torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False) # 1 x 1 conv
        self.bn1 = torch.nn.BatchNorm2d(out_channels)

        self.conv2 = torch.nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False) # 3 x 3 conv
        self.bn2 = torch.nn.BatchNorm2d(out_channels)

        self.conv3 = torch.nn.Conv2d(out_channels, out_channels*self.mul, kernel_size=1, stride=1, bias=False) # 1 x 1 conv
        self.bn3 = torch.nn.BatchNorm2d(out_channels*self.mul)


        self.shortcut = torch.nn.Sequential() # input 값이 그대로 반환됩니다.

        # shortcut connection: 입력 x의 크기를 맞추기 위해 1x1 컨볼루션 및 배치 정규화 수행
        # stride 같은 요소 때문에 출력의 크기가 달라질 수 있습니다.
        if stride != 1 or in_channels != out_channels*self.mul:
            self.shortcut = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels, out_channels*self.mul, kernel_size=1, stride=stride, bias=False), # 1 x 1 conv
                torch.nn.BatchNorm2d(out_channels*self.mul)
            )

    def forward(self, x):
        out = self.conv1(x) # [batch_size, out_channels, height, width]
        out = self.bn1(out) # [batch_size, out_channels, height, width]
        out = F.relu(out) # [batch_size, out_channels, height, width]
        out = self.conv2(out) # [batch_size, out_channels, height, width]
        out = self.bn2(out) # [batch_size, out_channels, height, width]
        out = F.relu(out) # [batch_size, out_channels, height, width]
        out = self.conv3(out) # [batch_size, out_channels*mul, height, width]
        out = self.bn3(out) # [batch_size, out_channels*mul, height, width]
        # skip connection
        out += self.shortcut(x) # [batch_size, out_channels*mul, height, width] or [batch_size, out_channels*mul*mul, height, width]
        out = F.relu(out)
        return out

In [7]:
class ResNet(torch.nn.Module):
    def __init__(self, block_mul, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        #RGB 3개채널에서 64개의 Kernel 사용
        self.in_channels = 64

        # Resnet 논문 구조 그대로 구현
        self.conv1 = torch.nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding = 3)
        self.bn1 = torch.nn.BatchNorm2d(self.in_channels)
        self.maxpool1 = torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self.make_layer(block_mul, 64, num_blocks[0], stride=1)
        self.layer2 = self.make_layer(block_mul, 128, num_blocks[1], stride=2)
        self.layer3 = self.make_layer(block_mul, 256, num_blocks[2], stride=2)
        self.layer4 = self.make_layer(block_mul, 512, num_blocks[3], stride=2)
        # Global Average Pooling 레이어 => 1차원 벡터로 만들기 위함
        # Global average pooling 참고자료: https://gaussian37.github.io/dl-concept-global_average_pooling/
        self.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))
        self.linear = torch.nn.Linear(512 * block_mul, num_classes)
        self.softmax = torch.nn.LogSoftmax(dim = 1)

    def make_layer(self, block_mul, out_channels, num_blocks, stride):
        # layer 앞부분에서만 크기를 절반으로 줄이므로, 아래와 같은 구조
        '''
        ex) stride = 2이고 num_blocks = 3인 경우, strides는 [2, 1, 1]로 생성되며,
        Bottleneck 블록들이 stride=2로 시작하여 앞부분에서 크기를 절반으로 줄이고, 나머지 블록들은 크기를 그대로 유지
        '''
        strides = [stride] + [1] * (num_blocks-1) # BottleNeck block의 stride 리스트 설정
        layers = []
        for i in range(num_blocks):
            layers.append(BottleNeck(self.in_channels, out_channels, strides[i], block_mul))
            self.in_channels = block_mul * out_channels
        return torch.nn.Sequential(*layers)

    def forward(self, x):
        '''
        INPUT :
            x = [batch_size, channel, height, width]
        OUTPUT :
            output = [batch_size, num_classes]
        '''
        out = self.conv1(x) # [batch_size, 64, height//2, width//2]
        out = self.bn1(out) # [batch_size, 64, height//2, width//2]
        out = F.relu(out) # [batch_size, 64, height//2, width//2]
        out = self.maxpool1(out) # [batch_size, 64, height//4, width//4]
        out = self.layer1(out) # [batch_size, 256, height//4, width//4]
        out = self.layer2(out) # [batch_size, 512, height//8, width//8]
        out = self.layer3(out) # [batch_size, 1024, height//16, width//16]
        out = self.layer4(out) # [batch_size, 2048, height//32, width//32]
        out = self.avgpool(out) # [batch_size, 2048, height//32, width//32]
        out = torch.flatten(out, 1) # [batch_size, 2048, 1, 1]
        out = self.linear(out) # [batch_size, 2048]
        out = self.softmax(out) # [batch_size, num_classes]
        return out

    def count_parameters(self):
            return sum(p.numel() for p in self.parameters() if p.requires_grad)

def ResNet50():
    return ResNet(4, [3, 4, 6, 3])

In [8]:
model = ResNet50()
print("ResNet50 total parameters: ", model.count_parameters())

ResNet50 total parameters:  23528586
